In [1]:
import pandas as pd
import zstandard

In [2]:
# sacadas de la catedra
def compress_size(bytefile):
    return len(zstandard.compress(bytefile))

def ncd(obj1, obj2):
    len1 = compress_size(obj1)
    len2 = compress_size(obj2)
    concat_len = len(zstandard.compress(obj1+obj2))
    return (concat_len-min(len1, len2))/max(len1, len2)

In [3]:
español = open("dHumanosEspañol.txt", "r", encoding='utf-8').read().encode()

Me fijo el tamaño antes de la compresion.

In [4]:
len(español)

12269

In [5]:
compressedEsp=compress_size(español)
compressedEsp

4661

In [6]:
portugues = open("dHumanosPortugues.txt", "r", encoding='utf-8').read().encode()


In [7]:
len(portugues)

12016

In [8]:
compressedPor = compress_size(portugues)
compressedPor

4694

In [9]:
ncd(español,español) #tiende a cero 

-0.013301866552242008

In [10]:
review = pd.read_csv("../review.csv", usecols=["text"])

In [11]:
review

,text
0,"If you decide to eat here, just be aware it is..."
1,Cute interior and owner (?) gave us tour of up...
2,I was really between 3 and 4 stars for this on...
3,First time there and it was excellent!!! It fe...
4,"Great burgers,fries and salad! Burgers have a..."
...,...
3838100,We redesigned my moms dress and mad it complet...
3838101,"This spot offers a great, affordable east week..."
3838102,This Home Depot won me over when I needed to g...
3838103,For when I'm feeling like ignoring my calorie-...


Para hacerlo un poco mas eficiente, me ahorro de hacer el compress del mismo archivo

In [12]:
def ncdModEspañol(review, español, tamaño):
    len1 = tamaño # me ahorro de hacer 3.800.000 de veces esto
    len2 = compress_size(review)
    concat_len = len(zstandard.compress(español+review))
    return (concat_len-min(len1, len2))/max(len1, len2)

def ncdModPortugues(review, portugues, tamaño):
    len1 = tamaño # me ahorro de hacer 3.800.000 de veces esto
    len2 = compress_size(review)
    concat_len = len(zstandard.compress(portugues+review))
    return (concat_len-min(len1, len2))/max(len1, len2)

In [13]:
import time

In [14]:
start = time.time()
review["NCD Español"] = review["text"].apply(lambda x : ncdModEspañol(x.encode(), español, compressedEsp))
end = time.time()
end-start

336.82652759552

5 minutos y pico

In [15]:
review["NCD Español"].describe()

count    3.838105e+06
mean     9.943669e-01
std      1.426005e-03
min      9.500107e-01
25%      9.935636e-01
50%      9.944218e-01
75%      9.950654e-01
max      1.091826e+00
Name: NCD Español, dtype: float64

El valor minimo es el que mas pudo comprimir los datos en ambos en uno solo, que serian que comparten las mismas palabras --> mismo idioma

In [16]:
review["NCD Español"].min()

0.9500107273117356

In [17]:
review[review["NCD Español"] == 0.9500107273117356].text.item()

'Mi experiencia fue malísima! Bastó una primera y última vez para saber que esta empresa trabaja sobre la base de la mentira y la falta de ética profesional!\n\nSólo los escogí porque me garantizaron entrega en 3 semanas. Yo tenía una fecha límite para hacer que llegaran esas cajas a Vzla. El señor que atiende este negocio me garantizó que serían entregadas antes de la fecha que yo le presenté (yo contaba con una semana extra para esperar). Fue así que envié las cajas con ellos. Los seguía por Facebook, y un día publicaron un vídeo con la leyenda "Así salen nuestras cajas de nuestra tienda, bla bla bla"  y, entre los lugares que mostraron, se encontraba el destino de mis cajas. Ante esto, le pregunté al dueño por WhatsApp si apenas para esa fecha estaban saliendo las cajas y, le recordé el estimado de tiempo que yo tenía. Pues, el "señor" me salió con la mala educación de decirme que para que mandé  con ellos, que cuál era la desconfianza, que cómo sabía yo que ahí iban mis cajas  y pa

In [18]:
review[review["NCD Español"] == 0.9500107273117356]

,text,NCD Español
1467338,Mi experiencia fue malísima! Bastó una primera...,0.950011


<h3>Ahora hago lo mismo para el idioma portugues</h3>

In [19]:
start = time.time()
review["NCD Portugues"] = review["text"].apply(lambda x : ncdModPortugues(x.encode(), portugues, compressedPor))
end = time.time()
end-start

344.78813219070435

In [20]:
review

,text,NCD Español,NCD Portugues
0,"If you decide to eat here, just be aware it is...",0.993135,0.993609
1,Cute interior and owner (?) gave us tour of up...,0.994207,0.996378
2,I was really between 3 and 4 stars for this on...,0.995495,0.996378
3,First time there and it was excellent!!! It fe...,0.995065,0.996378
4,"Great burgers,fries and salad! Burgers have a...",0.994636,0.996165
...,...,...,...
3838100,We redesigned my moms dress and mad it complet...,0.995495,0.996378
3838101,"This spot offers a great, affordable east week...",0.993564,0.995526
3838102,This Home Depot won me over when I needed to g...,0.992705,0.993396
3838103,For when I'm feeling like ignoring my calorie-...,0.996353,0.997444


In [21]:
review.sort_values(by="NCD Portugues").head(10)

,text,NCD Español,NCD Portugues
2327706,I have been on tours all over the world but th...,0.968891,0.977844
3680135,"Even if they cannot provide a service, and end...",0.968676,0.977844
2658552,After more than a decade of receiving care for...,0.971036,0.978483
3301116,Shame on you for dumping a couch into Darby cr...,0.966745,0.978483
760212,"I've honestly never written a negative review,...",0.968462,0.978696
1783841,"As a frequent traveler, I will say that the ma...",0.969534,0.978696
1423940,"I've been to Nashville several times, includin...",0.969749,0.978909
3599949,A Bad dentist and a Bad dentist office. As I ...,0.968462,0.979122
1522157,A fresh and healthy concept with dishes full o...,0.969105,0.979548
1377883,"Frustrated\n\nOn a Friday afternoon, three day...",0.969534,0.979548


Veo que en las 10 con el NCD mas bajo no hay ninguna en portugues, Pero puede pasar que las reviews no sean lo suficientemente largo y el ncd sea alto por esa razon, doy un ejemplo usando un fragmento del txt portugues. 

In [22]:
fragmento =  '''Todos os seres humanos nascem livres e iguais em dignidade e em direitos'''
len(fragmento)

72

In [23]:
ncd(fragmento.encode(), portugues) ## esto esta incluido en el txt, y aun asi es mas alto que el calculado

0.9853003834682573

voy a filtrar por reviews cortas

In [24]:

reviewsCortas = review[review["text"].str.len() < 200]

In [25]:
reviewsCortas.sort_values(by="NCD Portugues").head(5)

,text,NCD Español,NCD Portugues
1129380,Um dos lugares mais descolados que já conheci....,0.992276,0.990413
2406336,Preço justo. Comida boa e lugar bem típico ame...,0.991633,0.990413
780831,O calzone foi delicioso. Vc monta de acordo co...,0.992491,0.990626
301964,Excelente pedida! Escolhemos o lugar para o ja...,0.992276,0.990626
2346376,Ótimo pratos e apresentação porém o restaurant...,0.992276,0.990626
